# Introduction to network analytics with Python
### With applications in actuarial science

## Basic Network Features

### Random Network

We start with generating a random Barabási-Albert network to show how to construct such random networks. Afterwards, we analyse its properties and calculate some network features. This allows us to illustrate the meaning of the *scale-free* property. 

### Health Care Provider Network

## Representation Learning